In [1]:
import ee
from ee import batch

In [2]:
ee.Initialize()

## Monthly means for MACA data for national parks

In [3]:
maca = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA_MONTHLY')

In [4]:
parks = ee.FeatureCollection('users/majo3748/conus_natl_parks')

In [5]:
n_park = parks.size().getInfo()

In [6]:
park_list = parks.toList(n_park)

In [7]:
scenarios = ['historical', 'rcp85', 'rcp45']
variables = ['pr', 'tasmax', 'tasmin', 'huss', 'rsds', 'was']

Iterate over parks and scenarios, exporting park-wide means for each

In [8]:
processes = list()

In [9]:
for scenario in scenarios:
  scenario_data = maca.filterMetadata('scenario', 'equals', scenario)
  
  for var in variables:
    climate_var = scenario_data.select(var).toBands()
    
    for p in range(n_park):
      feat = ee.Feature(park_list.get(p))
      
      mean_dict = climate_var.reduceRegion(
        reducer = ee.Reducer.mean(), 
        geometry = feat.geometry(),
        scale = 4000, 
        maxPixels = 1e9
      )

      out = batch.Export.table.toDrive(
        collection = ee.FeatureCollection([ee.Feature(None, mean_dict)]),
        description = '_'.join([feat.get('UNIT_CO').getInfo(), var, scenario]), 
        folder = 'maca-monthly'
      )
      processes.append(batch.Task.start(out))

In [10]:
# how many jobs did we submit?
len(processes)

864